<a href="https://colab.research.google.com/github/rehmanaly0051/Exterior-Design-Model/blob/main/new_exde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/MyDrive/Exterior Design'

/content/drive/MyDrive/Exterior Design


In [4]:
!pip install diffusers transformers accelerate safetensors  opencv-python-headless --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.9 MB/s eta 0:00:00


In [ ]:
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu124

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
import cv2
import numpy as np
from PIL import Image
import random
import os

# Preprocess function
def preprocess_floor_plan(floor_plan_path, output_size=(768, 768)):
    """Process floor plan image for ControlNet input."""
    image = cv2.imread(floor_plan_path)
    if image is None:
        raise ValueError(f"Could not load image from {floor_plan_path}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Resize and pad
    h, w = image.shape[:2]
    ratio = min(output_size[0] / w, output_size[1] / h)
    new_size = (int(w * ratio), int(h * ratio))
    resized = cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)

    canvas = np.ones((output_size[1], output_size[0], 3), dtype=np.uint8) * 255
    offset_x = (output_size[0] - new_size[0]) // 2
    offset_y = (output_size[1] - new_size[1]) // 2
    canvas[offset_y:offset_y+new_size[1], offset_x:offset_x+new_size[0]] = resized

    gray = cv2.cvtColor(canvas, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)

    kernel = np.ones((2, 2), np.uint8)
    dilated_edges = cv2.dilate(edges, kernel, iterations=1)

    edge_image = cv2.cvtColor(dilated_edges, cv2.COLOR_GRAY2RGB)

    return Image.fromarray(edge_image)

# Load models
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny",
    torch_dtype=torch.float16,
    use_safetensors=True,
)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    use_safetensors=True,
)

# Setup pipeline
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
pipe.enable_xformers_memory_efficient_attention()

# Ask user for floor plan image
floor_plan_path = input("Enter the path to your floor plan image:\n")

# Preprocess the floor plan
control_image = preprocess_floor_plan(floor_plan_path)

# Define automatic prompts for different views
views = {
    "Front": "Front exterior view of a modern house based on the given floor plan, realistic, daytime",
    "Back": "Back exterior view of a modern house based on the given floor plan, realistic, daytime",
    "Left": "Left side exterior view of a modern house based on the given floor plan, realistic, daytime",
    "Right": "Right side exterior view of a modern house based on the given floor plan, realistic, daytime",
}

# Negative prompt to improve quality
negative_prompt = "low quality, blurry, bad architecture, distorted, ugly"

# Output size
single_view_size = (768, 768)

# Dictionary to store each generated image
generated_views = {}

# Generate images for each view
for view_name, view_prompt in views.items():
    seed = random.randint(0, 100000)
    generator = torch.manual_seed(seed)

    output = pipe(
        prompt=view_prompt,
        negative_prompt=negative_prompt,
        image=control_image,
        num_inference_steps=40,
        guidance_scale=7.5,
        generator=generator,
    )

    image = output.images[0]
    generated_views[view_name] = image

    print(f"✅ Generated {view_name} View")

# Create a 2x2 collage
collage_width = single_view_size[0] * 2
collage_height = single_view_size[1] * 2
collage = Image.new('RGB', (collage_width, collage_height), color=(255, 255, 255))

# Paste images into collage
collage.paste(generated_views["Front"], (0, 0))
collage.paste(generated_views["Back"], (single_view_size[0], 0))
collage.paste(generated_views["Left"], (0, single_view_size[1]))
collage.paste(generated_views["Right"], (single_view_size[0], single_view_size[1]))

# Save the final collage
final_output_path = "final_exterior_views.png"
collage.save(final_output_path)
print(f"\n🎯 All views combined and saved as {final_output_path} successfully!")

In [ ]:
import torch
print(torch.version.cuda)